In [ ]:
!pip install kfp google-cloud-pipeline-components --upgrade -q
!pip install google-cloud-aiplatform --upgrade -q
!pip install google-cloud-storage --upgrade -q

In [33]:
import kfp
from kfp import dsl
from kfp.dsl import Input, Output, Dataset, Model, Markdown

# Defina a imagem base que contém as bibliotecas necessárias.
BASE_IMAGE = 'python:3.9'
PACKAGES_TO_INSTALL = [
    'google-cloud-bigquery[pandas]',
    'pandas',
    'statsmodels',
    'scikit-learn',
    'db-dtypes',
    'tabulate',
    'google-cloud-aiplatform',
    'google-cloud-storage'
]

PROJECT_ID = "project-poc-purple"
BUCKET_NAME=f"gs://000-pipelines-root_{PROJECT_ID}"

# Create bucket
PIPELINE_ROOT = f"{BUCKET_NAME}/insurance_risk_pricing_pipeline/"

#=========================================================================================
# COMPONENTE 1: Carregar Dados do BigQuery
#=========================================================================================
@dsl.component(
    base_image=BASE_IMAGE,
    packages_to_install=PACKAGES_TO_INSTALL
)
def load_data_from_bq(
    project_id: str,
    location: str,
    bq_table: str,
    output_dataset: Output[Dataset]
):
    """Carrega dados de uma tabela do BigQuery e salva como um arquivo CSV."""
    from google.cloud import bigquery
    import pandas as pd

    client = bigquery.Client(project=project_id, location=location)

    print(f"Lendo dados da tabela {bq_table}...")
    sql_query = f"SELECT * FROM `{bq_table}`"
    df = client.query(sql_query).to_dataframe()

    print("Ajuste formato colunas")
    colunas_para_converter = df.select_dtypes(include='Int64').columns
    for col in colunas_para_converter:
        df[col] = df[col].astype('int64')

    print("✅ Sucesso! Dados carregados no DataFrame.")
    print(df.head())

    df.to_csv(output_dataset.path, index=False)
    print(f"Dataset salvo em: {output_dataset.path}")

#=========================================================================================
# COMPONENTE 2: Treinar Modelo de Frequência (Poisson)
#=========================================================================================
@dsl.component(
    base_image=BASE_IMAGE,
    packages_to_install=PACKAGES_TO_INSTALL
)
def train_frequency_model(
    input_dataset: Input[Dataset],
    target_column: str, # 'qtd_colisao_parcial' ou 'qtd_colisao_total'
    output_model: Output[Model]
):
    """Treina um modelo GLM Poisson para frequência de sinistros."""
    import pandas as pd
    import numpy as np
    import statsmodels.api as sm
    import joblib

    df = pd.read_csv(input_dataset.path)

    print(f"Treinando modelo de frequência para a coluna: {target_column}")

    features_freq = ['classe_bonus', 'idade', 'rns']
    X_freq = sm.add_constant(df[features_freq])

    model = sm.GLM(
        endog=df[target_column],
        exog=X_freq,
        family=sm.families.Poisson(),
        offset=np.log(df['exposicao'])
    )
    results = model.fit()

    joblib.dump(results, output_model.path)
    print(f"Modelo de frequência salvo em: {output_model.path}")
    print(results.summary())

#=========================================================================================
# COMPONENTE 3: Treinar Modelo de Severidade (Gamma)
#=========================================================================================
@dsl.component(
    base_image=BASE_IMAGE,
    packages_to_install=PACKAGES_TO_INSTALL
)
def train_severity_model(
    input_dataset: Input[Dataset],
    output_model: Output[Model]
):
    """Treina um modelo GLM Gamma para severidade de sinistros parciais."""
    import pandas as pd
    import statsmodels.api as sm
    import joblib

    df = pd.read_csv(input_dataset.path)

    print("Treinando modelo de severidade...")

    df_com_sinistro = df[df['valor_colisao_parcial'] > 0].copy()
    y_sev = df_com_sinistro['valor_colisao_parcial'] / df_com_sinistro['qtd_colisao_parcial']

    features_sev = ['classe_bonus', 'idade', 'rns', 'valor_veiculo']
    X_sev_com_sinistro = sm.add_constant(df_com_sinistro[features_sev])

    model = sm.GLM(
        endog=y_sev,
        exog=X_sev_com_sinistro,
        family=sm.families.Gamma(link=sm.families.links.Log())
    )
    results = model.fit()

    joblib.dump(results, output_model.path)
    print(f"Modelo de severidade salvo em: {output_model.path}")
    print(results.summary())

#=========================================================================================
# NOVO COMPONENTE 4: Avaliar e Registrar Modelo com Lógica Champion-Challenger
#=========================================================================================
@dsl.component(
    base_image=BASE_IMAGE,
    packages_to_install=PACKAGES_TO_INSTALL,
)
def evaluate_and_register_challenger(
    project_id: str,
    location: str,
    model_display_name: str,
    model_type: str, # 'frequency' or 'severity'
    target_column: str, # Nome da coluna alvo para o cálculo da métrica
    challenger_model: Input[Model],
    evaluation_dataset: Input[Dataset],
) -> str:
    """
    Usa um método de busca explícito e corrige o download do artefato do campeão.
    """
    import traceback
    try:
        import pandas as pd
        import numpy as np
        import statsmodels.api as sm
        import joblib
        from google.cloud import aiplatform, storage
        from google.api_core import exceptions
        import os

        print(f"--- INICIANDO AVALIAÇÃO PARA O MODELO: {model_display_name} ---")

        aiplatform.init(project=project_id, location=location)
        df = pd.read_csv(evaluation_dataset.path)

        def get_model_score(model_path: str) -> float:
            model = joblib.load(model_path)
            if model_type == 'frequency':
                if df['exposicao'].sum() == 0: return float('inf')
                features = ['classe_bonus', 'idade', 'rns']
                X = sm.add_constant(df[features])
                df['pred'] = model.predict(X)
                real_freq = df[target_column].sum() / df['exposicao'].sum()
                pred_freq = (df['pred'] * df['exposicao']).sum() / df['exposicao'].sum()
                return abs(real_freq - pred_freq)
            elif model_type == 'severity':
                df_claimed = df[df[target_column] > 0].copy()
                if df_claimed.empty: return float('inf')
                df_claimed = df_claimed[df_claimed['qtd_colisao_parcial'] > 0]
                if df_claimed.empty: return float('inf')
                real_sev_per_claim = df_claimed[target_column] / df_claimed['qtd_colisao_parcial']
                features = ['classe_bonus', 'idade', 'rns', 'valor_veiculo']
                X = sm.add_constant(df_claimed[features])
                pred_sev_per_claim = model.predict(X)
                return abs(real_sev_per_claim.mean() - pred_sev_per_claim.mean())
            return float('inf')

        challenger_score = get_model_score(challenger_model.path)
        print(f"Score do Challenger ({model_display_name}): {challenger_score}")

        champion_model = None
        champion_score = float('inf')
        champion_model_resource_name = ""
        model_list = []
        try:
            print(f"Buscando modelos com display name: '{model_display_name}'")
            model_list = aiplatform.Model.list(filter=f'display_name="{model_display_name}"')

            found_champion = False
            for model_version in model_list:
                if "default" in model_version.version_aliases:
                    champion_model = model_version
                    champion_model_resource_name = champion_model.resource_name
                    print(f"Versão default (campeã) encontrada: {champion_model_resource_name}")
                    found_champion = True
                    break

            if not found_champion:
                raise exceptions.NotFound(f"Nenhuma versão com o alias 'default' foi encontrada para o display name '{model_display_name}'.")

            local_champion_path = "/tmp/champion_model.joblib"
            storage_client = storage.Client(project=project_id)

            # --- CORREÇÃO FINAL: Manter o prefixo 'gs://' no URI ---
            model_uri = champion_model.uri
            if not model_uri.endswith('/'):
                model_uri += '/'
            blob = storage.Blob.from_string(model_uri + "model.joblib", client=storage_client)
            # --- FIM DA CORREÇÃO ---

            blob.download_to_filename(local_champion_path)
            print("Artefato do modelo campeão baixado com sucesso.")
            champion_score = get_model_score(local_champion_path)
            print(f"Score da versão default ({model_display_name}): {champion_score}")

        except exceptions.NotFound as e:
            print(f"{e} Challenger vence por padrão.")
        except Exception as e:
            print(f"Um erro ocorreu ao buscar/baixar a versão default. O challenger será considerado vencedor. Erro: {e}")

        if challenger_score < champion_score:
            print("Challenger é melhor. Promovendo para nova versão default.")
            gcs_uri = challenger_model.uri
            path_parts = gcs_uri.replace("gs://", "").split('/')
            bucket_name = path_parts[0]
            object_name = '/'.join(path_parts[1:])
            object_dir = os.path.dirname(object_name)
            artifact_uri_for_upload = f"gs://{bucket_name}/{object_dir}"
            new_object_name = f"{object_dir}/model.joblib"
            storage_client = storage.Client(project=project_id)
            source_bucket = storage_client.bucket(bucket_name)
            source_blob = source_bucket.blob(object_name)
            source_bucket.copy_blob(source_blob, source_bucket, new_object_name)

            parent_model_ref = None
            if model_list:
                parent_model_ref = model_list[0].resource_name.split('@')[0]
                print(f"Modelo pai encontrado para versionamento: {parent_model_ref}")

            new_champion = aiplatform.Model.upload(
                display_name=model_display_name,
                artifact_uri=artifact_uri_for_upload,
                serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-1:latest",
                parent_model=parent_model_ref,
                is_default_version=True
            )
            new_champion.wait()
            print(f"Upload bem-sucedido. Nova versão default: {new_champion.resource_name} (Versão: {new_champion.version_id})")

            print(f"--- AVALIAÇÃO CONCLUÍDA. NOVO DEFAULT: {new_champion.resource_name} ---")
            return new_champion.resource_name
        else:
            print("Versão default atual é melhor ou igual. Mantendo a versão existente.")
            print(f"--- AVALIAÇÃO CONCLUÍDA. DEFAULT MANTIDO: {champion_model_resource_name} ---")
            return champion_model_resource_name

    except Exception as e:
        print("!!!!!!!!!! ERRO FATAL NO COMPONENTE !!!!!!!!!!")
        traceback.print_exc()
        raise e

#=========================================================================================
# COMPONENTE 5: Predizer e Calcular Prêmio
#=========================================================================================
@dsl.component(
    base_image=BASE_IMAGE,
    packages_to_install=PACKAGES_TO_INSTALL
)
def predict_from_registered_models(
    project_id: str,
    location: str,
    input_dataset: Input[Dataset],
    model_freq_parcial_resource_name: str,
    model_freq_total_resource_name: str,
    model_sev_parcial_resource_name: str,
    output_predictions: Output[Dataset]
):
    """Carrega modelos registrados, aplica predições e calcula o prêmio."""
    import pandas as pd
    import numpy as np
    import joblib
    import statsmodels.api as sm
    from google.cloud import aiplatform, storage
    import os

    df = pd.read_csv(input_dataset.path)
    aiplatform.init(project=project_id, location=location)

    def download_and_load_model(model_resource_name: str) -> any:
        """Baixa um modelo via Model Registry e o carrega."""
        print(f"Carregando modelo do resource name: {model_resource_name}")
        model = aiplatform.Model(model_name=model_resource_name)
        model_artifact_dir = model.uri
        print(f"URI do artefato do modelo: {model_artifact_dir}")

        # Nome do arquivo de modelo padrão salvo pelo Vertex AI
        model_file_name = 'model.joblib'
        # Caso o upload tenha sido de uma versão mais antiga da plataforma, pode ser 'model.pkl'
        # Esta lógica tenta encontrar o arquivo de modelo correto.
        storage_client = storage.Client(project=project_id)
        bucket_name = model_artifact_dir.split('/')[2]
        prefix = '/'.join(model_artifact_dir.split('/')[3:]) + '/'
        blobs = storage_client.list_blobs(bucket_name, prefix=prefix)
        model_blob_name = None
        for blob in blobs:
            if blob.name.endswith('.joblib') or blob.name.endswith('.pkl'):
                model_blob_name = blob.name
                break

        if not model_blob_name:
            raise FileNotFoundError(f"Não foi possível encontrar um arquivo .joblib ou .pkl em {model_artifact_dir}")

        local_model_file = f"/tmp/{model_resource_name.split('/')[-1]}.joblib"

        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(model_blob_name)
        blob.download_to_filename(local_model_file)
        print(f"Modelo baixado para {local_model_file}")

        return joblib.load(local_model_file)

    results_freq_parcial = download_and_load_model(model_freq_parcial_resource_name)
    results_freq_total = download_and_load_model(model_freq_total_resource_name)
    results_sev_parcial = download_and_load_model(model_sev_parcial_resource_name)
    print("Modelos carregados com sucesso do Registry.")

    df['freq_parcial_pred'] = results_freq_parcial.predict(sm.add_constant(df[['classe_bonus', 'idade', 'rns']]))
    df['freq_total_pred'] = results_freq_total.predict(sm.add_constant(df[['classe_bonus', 'idade', 'rns']]))
    df['sev_parcial_pred'] = results_sev_parcial.predict(sm.add_constant(df[['classe_bonus', 'idade', 'rns', 'valor_veiculo']]))

    df['premio_risco_pred'] = (df['freq_parcial_pred'] * df['sev_parcial_pred'] + df['freq_total_pred'] * df['valor_veiculo'])

    print("Cálculos finalizados. Salvando predições.")
    df.to_csv(output_predictions.path, index=False)
    print(df[['valor_veiculo', 'freq_parcial_pred', 'sev_parcial_pred', 'freq_total_pred', 'premio_risco_pred']].head())

#=========================================================================================
# COMPONENTE 6: Gerar Relatório de Sumarização (sem alterações)
#=========================================================================================
@dsl.component(
    base_image=BASE_IMAGE,
    packages_to_install=PACKAGES_TO_INSTALL
)
def generate_summary_report(
    predictions_dataset: Input[Dataset],
    output_report: Output[Markdown]
):
    """Calcula métricas sumarizadas e gera um relatório de comparação."""
    import pandas as pd
    import numpy as np
    df = pd.read_csv(predictions_dataset.path)

    soma_exposicao = df['exposicao'].sum()
    soma_qtd_parcial = df['qtd_colisao_parcial'].sum()
    soma_valor_parcial = df['valor_colisao_parcial'].sum()
    soma_qtd_total = df['qtd_colisao_total'].sum()

    freq_parcial_real = soma_qtd_parcial / soma_exposicao
    sev_parcial_real = soma_valor_parcial / soma_qtd_parcial if soma_qtd_parcial > 0 else 0
    freq_total_real = soma_qtd_total / soma_exposicao

    premio_risco_real = freq_parcial_real * sev_parcial_real + freq_total_real * np.average(df['valor_veiculo'], weights=df['exposicao'])
    premio_risco_prev_total = (df['premio_risco_pred'] * df['exposicao']).sum()
    premio_risco_prev_medio = premio_risco_prev_total / soma_exposicao

    soma_qtd_parcial_pred = (df['freq_parcial_pred'] * df['exposicao']).sum()
    soma_valor_parcial_pred = (df['freq_parcial_pred'] * df['exposicao'] * df['sev_parcial_pred']).sum()
    soma_qtd_total_pred = (df['freq_total_pred'] * df['exposicao']).sum()

    freq_parcial_prev = soma_qtd_parcial_pred / soma_exposicao
    sev_parcial_prev = soma_valor_parcial_pred / soma_qtd_parcial_pred if soma_qtd_parcial_pred > 0 else 0
    freq_total_prev = soma_qtd_total_pred / soma_exposicao
    premio_risco_prev = freq_parcial_prev * sev_parcial_prev + freq_total_prev * np.average(df['valor_veiculo'], weights=df['exposicao'])

    sumario = pd.DataFrame({
        'Métrica': ['Frequência de Colisão Parcial', 'Severidade de Colisão Parcial', 'Frequência de Colisão Total', 'Prêmio de Risco (Puro Médio)'],
        'Valor Realizado': [freq_parcial_real, sev_parcial_real, freq_total_real, premio_risco_real],
        'Valor Previsto pelo Modelo': [freq_parcial_prev, sev_parcial_prev, freq_total_prev, premio_risco_prev]
    })
    markdown_report = "# Relatório de Comparação Real x Previsto\n\n" + sumario.to_markdown(index=False)
    with open(output_report.path, 'w') as f:
        f.write(markdown_report)
    print("Relatório gerado com sucesso!")
    print("RELATÓRIO DE COMPARAÇÃO REAL X PREVISTO")
    print("="*75)
    print(sumario.to_string(index=False))
    print("="*75)

#=========================================================================================
# DEFINIÇÃO DA PIPELINE ATUALIZADA COM LÓGICA CHAMPION-CHALLENGER
#=========================================================================================
@dsl.pipeline(
    pipeline_root=PIPELINE_ROOT,
    name='pipeline-precificacao-risco-champion-challenger',
    description='Treina, avalia (champion-challenger) e registra modelos para calcular prêmio de risco.'
)
def risk_pricing_pipeline_champion_flow(
    project_id: str = 'project-poc-purple',
    location: str = 'us-central1',
    bq_location: str = 'US',
    bq_table: str = 'project-poc-purple.demos.dados_apolices_v1',
    model_name_prefix: str = 'risk-pricing-ins-v3'
):
    load_task = load_data_from_bq(
        project_id=project_id, location=bq_location, bq_table=bq_table
    )

    # --- Trilha Frequência Parcial ---
    train_freq_parcial_task = train_frequency_model(
        input_dataset=load_task.outputs['output_dataset'], target_column='qtd_colisao_parcial'
    )

    eval_freq_parcial_task = evaluate_and_register_challenger(
        project_id=project_id, location=location,
        model_display_name=f"{model_name_prefix}-freq-parcial",
        model_type='frequency', target_column='qtd_colisao_parcial',
        challenger_model=train_freq_parcial_task.outputs['output_model'],
        evaluation_dataset=load_task.outputs['output_dataset']
    )

    # --- Trilha Frequência Total ---
    train_freq_total_task = train_frequency_model(
        input_dataset=load_task.outputs['output_dataset'], target_column='qtd_colisao_total'
    )

    eval_freq_total_task = evaluate_and_register_challenger(
        project_id=project_id, location=location,
        model_display_name=f"{model_name_prefix}-freq-total",
        model_type='frequency', target_column='qtd_colisao_total',
        challenger_model=train_freq_total_task.outputs['output_model'],
        evaluation_dataset=load_task.outputs['output_dataset']
    )

    # --- Trilha Severidade Parcial ---
    train_sev_parcial_task = train_severity_model(
        input_dataset=load_task.outputs['output_dataset']
    )

    eval_sev_parcial_task = evaluate_and_register_challenger(
        project_id=project_id, location=location,
        model_display_name=f"{model_name_prefix}-sev-parcial",
        model_type='severity', target_column='valor_colisao_parcial',
        challenger_model=train_sev_parcial_task.outputs['output_model'],
        evaluation_dataset=load_task.outputs['output_dataset']
    )

    # --- Etapa de Predição usando os modelos campeões ---
    predict_task = predict_from_registered_models(
        project_id=project_id, location=location,
        input_dataset=load_task.outputs['output_dataset'],
        model_freq_parcial_resource_name=eval_freq_parcial_task.output,
        model_freq_total_resource_name=eval_freq_total_task.output,
        model_sev_parcial_resource_name=eval_sev_parcial_task.output
    )

    # --- Etapa Final de Geração de Relatório ---
    report_task = generate_summary_report(
        predictions_dataset=predict_task.outputs['output_predictions']
    )

In [34]:
#=========================================================================================
# COMPILAÇÃO DA PIPELINE
#=========================================================================================
from kfp.compiler import Compiler

PIPELINE_FILE = 'risk_pricing_champion_challenger.yaml'

Compiler().compile(
    pipeline_func=risk_pricing_pipeline_champion_flow,
    package_path=PIPELINE_FILE
)
print(f"Pipeline compilada com sucesso para '{PIPELINE_FILE}'")

Pipeline compilada com sucesso para 'risk_pricing_champion_challenger.yaml'


In [35]:
#=========================================================================================
# EXECUÇÃO DA PIPELINE
#=========================================================================================
from google.cloud import aiplatform

PROJECT_ID = "project-poc-purple"
REGION = "us-central1"
PIPELINE_FILE = 'risk_pricing_champion_challenger.yaml'

aiplatform.init(project=PROJECT_ID, location=REGION)

job = aiplatform.PipelineJob(
  display_name="risk-pricing-champion-challenger-pipeline",
  template_path=PIPELINE_FILE,
  enable_caching=True,
  location=REGION
)

job.run()